In [1]:
%pip install ngboost
%pip install "xgboost>=2.0.0"
try:
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
    HAVE_NGBOOST = True
except Exception:
    HAVE_NGBOOST = False

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 7.9 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=81e7d317855916f118ad9cea81383e7281588ae746a6695b606d8ea6f51d7e7f
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [6]:
# agrega_resultados_por_features_com_logs.py
import os
import numpy as np
import pandas as pd
from time import time

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [7]:
# ====== Config simples de log ======
VERBOSE = True          # imprime tudo
PRINT_EVERY = 1         # imprimir a cada N runs (1 = toda run)
N_RUNS = 20
SEED_BASE = 42          # semente base para reprodutibilidade do fluxo de sementes

# ====== Dados ======
df = pd.read_csv('dados_2h_aggregado_final.csv')
df['Data'] = pd.to_datetime(df['Data'])

features1 = ['TO_AI6401_01', 'TO_AI6402_01', 'TO_BO6311_EST1', 'TO_DI6311_01', 'TO_FI6311_01',
             'TO_FI8271_01', 'TO_FY6104_01', 'TO_LI6401_01', 'TO_LI6401_02', 'TO_LI6402_01',
             'TO_LI6411_01', 'TO_LI6411_02', 'TO_LI6412_01', 'TO_LI6412_02', 'TO_LI6431_01',
             'TO_LI6432_01', 'TO_MF6401_M1_EST1', 'TO_MF6402_M1_EST1', 'TO_MF6411_M1_EST1',
             'TO_MF6412_M1_EST1', 'TO_MF6421_M1_EST1', 'TO_PHY4804_02', 'TO_PHY4814_02', 'TO_SIMF640101',
             'TO_SIMF640102', 'TO_SIMF640201', 'TO_SIMF640202', 'TO_SIMF641101', 'TO_SIMF641102',
             'TO_SIMF641201', 'TO_SIMF641202', 'TO_SIMF6421', 'TO_SIMF6422', 'TO_SIMF6431', 'TO_SIMF6432', 'TO_WI4804_01']
features2 = ['TO_WI4804_01', 'TO_PHY4814_02', 'TO_PHY4804_02', 'TO_SIMF641201', 'TO_LI6412_01', 'TO_SIMF640202', 'TO_MF6401_M1_EST1', 'TO_SIMF6422', 'TO_SIMF641102', 'TO_FI8271_01']
features3 = ['TO_MF6412_M1_EST1', 'TO_FI8271_01', 'TO_PHY4814_02', 'TO_WI4804_01', 'TO_SIMF6421', 'TO_FI6311_01', 'TO_PHY4804_02', 'TO_SIMF6431', 'TO_SIMF640202', 'TO_LI6411_01']
features4 = ['TO_PHY4804_02', 'TO_AI6401_01', 'TO_AI6402_01', 'TO_LI6412_01', 'TO_LI6412_02',
             'TO_LI6431_01', 'TO_FI8271_01', 'TO_PHY4814_02', 'TO_FY6104_01', 'TO_WI4804_01',
             'TO_SIMF640101', 'TO_SIMF640201', 'TO_SIMF640202', 'TO_SIMF641101', 'TO_SIMF641102',
             'TO_SIMF641201', 'TO_SIMF641202', 'TO_SIMF6421', 'TO_SIMF6422', 'TO_SIMF6431']



In [ ]:
%%writefile Prediction_intervals.py
# -*- coding: utf-8 -*-
"""
Prediction_intervals.py
------------------
Script para construção de intervalos de predição com múltiplos métodos.

Uso (variáveis de ambiente):
- DATA_FILE: caminho do CSV (default: dados_2h_aggregado_final.csv)
- SPLIT_TYPE: "random" | "temporal"
- MODEL_NAME: "xgb" | "lgbm" | "hgb" (default: xgb)
- INTERVAL_METHOD: ver lista em METHOD_CHOICES
- ALPHA: nível (default: 0.05)
- SEED: semente
- USE_ZSCORE_OUTLIERS: 0/1
- ZS_THRESH: 3.0
- CONSERVATIVE_Q: 0/1 (default: 1)
- USE_POINT_MODEL: 0/1
"""
import os, math, random, inspect
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor

# backends
HAVE_XGB = True
try:
    from xgboost import XGBRegressor
except Exception:
    HAVE_XGB = False

HAVE_LGBM = True
try:
    from lightgbm import LGBMRegressor
except Exception:
    HAVE_LGBM = False

# NGBoost
HAVE_NGB = True
try:
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
except Exception:
    HAVE_NGB = False

import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# CONFIG
# =========================
CSV_FILE   = os.environ.get("DATA_FILE", "dados_2h_aggregado_final.csv")
SPLIT_TYPE = os.environ.get("SPLIT_TYPE", "random").lower()          # 'random' | 'temporal'
MODEL_NAME = os.environ.get("MODEL_NAME", "xgb").lower()             # 'xgb' | 'lgbm' | 'hgb'
METHOD     = os.environ.get("INTERVAL_METHOD", "enbpi_rf").lower()
ALPHA      = float(os.environ.get("ALPHA", "0.05"))
SEED       = int(os.environ.get("SEED",  "42"))

USE_ZSCORE_OUTLIERS = bool(int(os.environ.get("USE_ZSCORE_OUTLIERS", "0")))  # 1/0
ZS_THRESH           = float(os.environ.get("ZS_THRESH", "3.0"))

# quantil conservador (garantia finita-amostral) vs linear
CONSERVATIVE_Q = bool(int(os.environ.get("CONSERVATIVE_Q", "1")))

# por padrão NÃO usar um modelo separado para ponto (evita “apertar” métricas com cal)
USE_SEPARATE_POINT_MODEL = bool(int(os.environ.get("USE_POINT_MODEL", "0")))

# Semente global
np.random.seed(SEED)
random.seed(SEED)

RESULTS_DIR = Path("results"); RESULTS_DIR.mkdir(exist_ok=True)

FEATURES = ['TO_PHY4804_02', 'TO_AI6401_01', 'TO_AI6402_01', 'TO_LI6412_01', 'TO_LI6412_02',
             'TO_LI6431_01', 'TO_FI8271_01', 'TO_PHY4814_02', 'TO_FY6104_01', 'TO_WI4804_01',
             'TO_SIMF640101', 'TO_SIMF640201', 'TO_SIMF640202', 'TO_SIMF641101', 'TO_SIMF641102',
             'TO_SIMF641201', 'TO_SIMF641202', 'TO_SIMF6421', 'TO_SIMF6422', 'TO_SIMF6431']

TARGET = 'P_CONFLTTO_QQ_GLOBAL_SIO2'

# Paleta
PALETTE = sns.color_palette("pastel")
COLOR_REAL = PALETTE[0]; COLOR_PRED = PALETTE[1]; COLOR_FILL = PALETTE[2]

# =========================
# Utils
# =========================
def rmse(y_true, y_pred): return math.sqrt(mean_squared_error(y_true, y_pred))

def q_conformal(scores: np.ndarray, alpha: float, conservative: bool = True) -> float:
    s = np.asarray(scores).ravel()
    n = len(s)
    if n == 0: raise ValueError("scores vazios.")
    if conservative:
        from math import ceil
        k = ceil((n + 1) * (1 - alpha))
        k = min(max(1, k), n)
        return np.partition(s, k-1)[k-1]
    else:
        return np.quantile(s, 1 - alpha, method="linear")

def get_point_model(name: str, random_state: int = 42):
    name = name.lower()
    if name == "xgb" and HAVE_XGB:
        # ==== seus hiperparâmetros do XGB para modelos de ponto ====
      return XGBRegressor(
    n_estimators=1723,
    learning_rate=0.037956993618682176,
    max_depth=7,
    subsample=0.7836258304564441,
    colsample_bytree=0.9491409614285389,
    min_child_weight=3.7584550421359695,
    reg_lambda=15.78535411656014,
    reg_alpha=2.267819381076329,
    gamma=0.3719023165905777,
    tree_method="hist",
    eval_metric="rmse",
    random_state=random_state,
    n_jobs=-1,
)

    if name == "lgbm" and HAVE_LGBM:
        return LGBMRegressor(
            n_estimators=600, learning_rate=0.05,
            num_leaves=2**12, max_depth=12, min_data_in_leaf=20,
            subsample=0.8, colsample_bytree=0.8,
            random_state=random_state, verbose=-1
        )

    from sklearn.ensemble import HistGradientBoostingRegressor
    return HistGradientBoostingRegressor(random_state=random_state)

def ensure_numeric(df: pd.DataFrame) -> pd.DataFrame:
    return df.apply(pd.to_numeric, errors="coerce")

def split_data(X: pd.DataFrame, y: pd.Series, split_type: str, seed: int,
               train_frac=0.80, cal_frac=0.10):
   
    n = len(X)
    n_train = int(n * train_frac)  
    n_cal = int(n * cal_frac)      
    n_test = n - n_train - n_cal   

    if split_type == "temporal":
        X_train = X.iloc[:n_train]; y_train = y.iloc[:n_train]
        X_cal   = X.iloc[n_train:n_train + n_cal]; y_cal = y.iloc[n_train:n_train + n_cal]
        X_test  = X.iloc[n_train + n_cal:]; y_test = y.iloc[n_train + n_cal:]
        return X_train, X_cal, X_test, y_train, y_cal, y_test


    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=1.0 - train_frac,
                                               shuffle=True, random_state=seed)
    X_tr, X_ca, y_tr, y_ca = train_test_split(X_tr, y_tr, test_size=cal_frac / (train_frac),
                                               shuffle=True, random_state=seed + 1)
    return X_tr, X_ca, X_te, y_tr, y_ca, y_te

# =========================
# Intervalos canônicos
# =========================
def t_student_intervals(y_pred, residuals, n_train, p, alpha=ALPHA):
    """Intervalo aproximado usando t-Student sobre resíduos de treino."""
    s_err = np.std(residuals, ddof=1)
    dof = max(n_train - p, 1)
    t_val = stats.t.ppf(1 - alpha/2, dof)
    margin = t_val * s_err * math.sqrt(1 + 1.0/n_train)
    return y_pred - margin, y_pred + margin, dict(s_err=s_err, dof=dof, t_val=t_val, margin=margin)

def full_conformal_cvplus(model, X_train_cal, y_train_cal, X_test, alpha=ALPHA, folds=5, seed=SEED):
    """Full conformal com resíduos out-of-fold (CV+)."""
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    preds_oof = np.zeros(len(X_train_cal))
    for tr_idx, va_idx in kf.split(X_train_cal):
        m = clone(model)
        m.fit(X_train_cal.iloc[tr_idx], y_train_cal.iloc[tr_idx])
        preds_oof[va_idx] = m.predict(X_train_cal.iloc[va_idx])
    scores = np.abs(y_train_cal.values - preds_oof)
    q_hat = q_conformal(scores, alpha, conservative=CONSERVATIVE_Q)
    m_final = clone(model); m_final.fit(X_train_cal, y_train_cal)
    y_pred = m_final.predict(X_test)
    return y_pred, y_pred - q_hat, y_pred + q_hat, dict(q_hat=q_hat)

def full_conformal_prediction_intervalsF(model, X_train, X_test, y_train, alpha=0.05):
    """
    Variante "F": Full Conformal Prediction simples (sem CV+), usando o quantil
    ((n + 1) * (1 - alpha))/n aplicado aos resíduos absolutos de treino.
    """
    m = clone(model); m.fit(X_train, y_train)
    y_pred_train = m.predict(X_train)
    residuals_train = np.abs(y_train - y_pred_train)
    n = len(X_train)
    q_quantile = (n + 1) * (1 - alpha) / n
    q_ = np.quantile(residuals_train, q_quantile, method="linear")
    y_pred_test = m.predict(X_test)
    lower_bound = y_pred_test - q_
    upper_bound = y_pred_test + q_
    return y_pred_test, lower_bound, upper_bound, dict(q_hat=float(q_))

def split_conformal_intervals(model, X_train, y_train, X_cal, y_cal, X_test, alpha=ALPHA):
    """Split conformal canônico: mesmo modelo para cal e teste."""
    m = clone(model)
    m.fit(X_train, y_train)
    y_cal_pred = m.predict(X_cal)
    scores = np.abs(y_cal.values - y_cal_pred)
    q_hat = q_conformal(scores, alpha, conservative=CONSERVATIVE_Q)
    y_pred = m.predict(X_test)
    return y_pred, y_pred - q_hat, y_pred + q_hat, dict(q_hat=q_hat)



def bootstrap_residual_intervals(model, X_train, y_train, X_cal, y_cal, X_test,
                                 n_bootstrap=1000, alpha=ALPHA, seed=SEED):
    """
    Bootstrap de resíduos (percentil):
      - Treina no TRAIN
      - Calcula resíduos NO CAL (out-of-sample)
      - Reamostra resíduos do CAL e os adiciona às predições de teste
    """
    rng = np.random.default_rng(seed)
    m = clone(model)
    m.fit(X_train, y_train)

    # predições
    y_pred_test = m.predict(X_test)
    y_pred_cal  = m.predict(X_cal)

    # resíduos OUT-OF-SAMPLE (calibração)
    res_cal = (y_cal.values if hasattr(y_cal, "values") else np.asarray(y_cal)) - y_pred_cal

    # recentrar resíduos (garante média ~0)
    res_cal = res_cal - np.mean(res_cal)

    n_obs = len(X_test)
    preds = np.empty((n_bootstrap, n_obs), dtype=float)
    for i in range(n_bootstrap):
        boot_res = rng.choice(res_cal, size=n_obs, replace=True)
        preds[i, :] = y_pred_test + boot_res

    lo = np.percentile(preds, 100 * (alpha / 2.0), axis=0)
    up = np.percentile(preds, 100 * (1 - alpha / 2.0), axis=0)
    return y_pred_test, lo, up, dict(n_bootstrap=n_bootstrap, seed=seed)

def bootstrap_prediction_intervalsF(model, X_train, y_train, X_test, n_bootstrap=1000, alpha=0.05, seed=SEED):
    """Variante 'F' do bootstrap utilizando sklearn.utils.resample."""
    m = clone(model); m.fit(X_train, y_train)
    y_pred_test = m.predict(X_test)
    residuals = y_train - m.predict(X_train)
    n_obs = len(X_test)
    bootstrap_predictions = np.zeros((n_bootstrap, n_obs))
    rng = np.random.default_rng(seed)
    for i in range(n_bootstrap):
        idx = rng.integers(0, len(residuals), size=n_obs)
        bootstrap_residuals = residuals.values[idx] if hasattr(residuals, "values") else residuals[idx]
        bootstrap_predictions[i, :] = y_pred_test + bootstrap_residuals
    lower_bound = np.percentile(bootstrap_predictions, 100 * alpha / 2, axis=0)
    upper_bound = np.percentile(bootstrap_predictions, 100 * (1 - alpha / 2), axis=0)
    return y_pred_test, lower_bound, upper_bound, dict(n_bootstrap=n_bootstrap, seed=seed)

d
def pi_cqr(X_tr, y_tr, X_cal, y_cal, X_te, alpha=ALPHA, seed=SEED):
    """CQR usando GradientBoostingRegressor (quantile)."""
    lo = GradientBoostingRegressor(loss='quantile', alpha=alpha/2, n_estimators=400, random_state=seed)
    hi = GradientBoostingRegressor(loss='quantile', alpha=1 - alpha/2, n_estimators=400, random_state=seed)
    lo.fit(X_tr, y_tr); ql_cal = lo.predict(X_cal)
    hi.fit(X_tr, y_tr); qh_cal = hi.predict(X_cal)
    scores = np.maximum(ql_cal - y_cal.values, y_cal.values - qh_cal)
    qval = q_conformal(scores, alpha, conservative=CONSERVATIVE_Q)
    X_tc = pd.concat([X_tr, X_cal], axis=0); y_tc = pd.concat([y_tr, y_cal], axis=0)
    lo.fit(X_tc, y_tc); ql_te = lo.predict(X_te)
    hi.fit(X_tc, y_tc); qh_te = hi.predict(X_te)
    point_pred = 0.5*(ql_te + qh_te)
    return point_pred, ql_te - qval, qh_te + qval, dict(q_hat=qval)


def pi_enbpi(X_tr, y_tr, X_cal, y_cal, X_te, alpha=ALPHA, seed=SEED):
    return pi_enbpiF(X_tr, y_tr, X_cal, y_cal, X_te, alpha=alpha, seed=seed)

def pi_ngboostF(X_tr, y_tr, X_te, alpha=ALPHA, seed=SEED):
    """NGBoost (Normal). Intervalo direto via PPF do Normal previsto."""
    if not HAVE_NGB:
        raise RuntimeError("NGBoost não disponível.")
    ngb = NGBRegressor(Dist=Normal, n_estimators=600, learning_rate=0.03, random_state=seed)
    ngb.fit(X_tr, y_tr)
    dist = ngb.pred_dist(X_te)
    mu = dist.loc
    lo = dist.ppf(alpha/2)
    up = dist.ppf(1 - alpha/2)
    return mu, lo, up, dict()

def pi_ngboost(X_tr, y_tr, X_te, alpha=ALPHA, seed=SEED):
    return pi_ngboostF(X_tr, y_tr, X_te, alpha=alpha, seed=seed)

# ====
# ===== CQR com XGBoost (quantile) e EnbPI com XGB =====

# Seus hípers do XGB — separando para ponto (rmse) e para quantil
XGB_PARAMS_POINT = dict(
    n_estimators=1723,
    learning_rate=0.037956993618682176,
    max_depth=7,
    subsample=0.7836258304564441,
    colsample_bytree=0.9491409614285389,
    min_child_weight=3.7584550421359695,
    reg_lambda=15.78535411656014,
    reg_alpha=2.267819381076329,
    gamma=0.3719023165905777,
    tree_method="hist",
    eval_metric="rmse",
    n_jobs=-1,
)

XGB_PARAMS_QUANTILE = dict(
    n_estimators=1723,
    learning_rate=0.037956993618682176,
    max_depth=7,
    subsample=0.7836258304564441,
    colsample_bytree=0.9491409614285389,
    min_child_weight=3.7584550421359695,
    reg_lambda=15.78535411656014,
    reg_alpha=2.267819381076329,
    gamma=0.3719023165905777,
    tree_method="hist",
    n_jobs=-1,
)


def pi_cqr_xgb(
    X_tr, y_tr, X_cal, y_cal, X_te,
    alpha=ALPHA, seed=SEED,
    xgb_params=None, use_q50_point=True,
    early_stopping=True, es_val_frac=0.2, es_rounds=50
):
    if not HAVE_XGB:
        raise RuntimeError("XGBoost não disponível.")
    from xgboost import XGBRegressor

    # defaults internos; serão sobrescritos por xgb_params
    base_def = dict(
        subsample=1.0, reg_lambda=1.0, reg_alpha=10.0, min_child_weight=3.0,
        max_depth=6, learning_rate=0.01, gamma=0.0, colsample_bytree=0.6,
        n_estimators=300, n_jobs=-1, tree_method="hist", random_state=seed
    )
    if xgb_params:
        base_def.update(xgb_params)

    q_lo, q_hi = alpha/2.0, 1.0 - alpha/2.0

    # split interno para early stopping (não usa calibração)
    if early_stopping:
        X_tr_in, X_val, y_tr_in, y_val = train_test_split(
            X_tr, y_tr, test_size=es_val_frac, random_state=seed, shuffle=True
        )
    else:
        X_tr_in, y_tr_in = X_tr, y_tr
        X_val, y_val = None, None

    # ES: usa callback se disponível (XGB>=2.0); senão, early_stopping_rounds
    def _fit_quantile(model, X, y, Xv=None, yv=None):
        sig = inspect.signature(model.fit)
        kwargs = {}
        if Xv is not None and yv is not None and "eval_set" in sig.parameters:
            kwargs["eval_set"] = [(Xv, yv)]
            try:
                from xgboost.callback import EarlyStopping as XGB_EarlyStopping
                if "callbacks" in sig.parameters:
                    kwargs["callbacks"] = [XGB_EarlyStopping(rounds=es_rounds, save_best=True, maximize=False)]
                elif "early_stopping_rounds" in sig.parameters:
                    kwargs["early_stopping_rounds"] = es_rounds
            except Exception:
                if "early_stopping_rounds" in sig.parameters:
                    kwargs["early_stopping_rounds"] = es_rounds
        if "verbose" in sig.parameters:
            kwargs["verbose"] = False

        model.fit(X, y, **kwargs)

        best_it = getattr(model, "best_iteration", None)
        if best_it is None:
            try:
                best_it = model.get_booster().best_iteration
            except Exception:
                best_it = None
        if best_it is None:
            best_it = base_def["n_estimators"]
        return int(best_it)

    # modelos de quantil low/high
    lo = XGBRegressor(objective="reg:quantileerror", quantile_alpha=q_lo, **base_def)
    hi = XGBRegressor(objective="reg:quantileerror", quantile_alpha=q_hi, **base_def)

    n_lo = _fit_quantile(lo, X_tr_in, y_tr_in, X_val, y_val)
    n_hi = _fit_quantile(hi, X_tr_in, y_tr_in, X_val, y_val)

    # não-conformidades na calibração
    ql_cal = lo.predict(X_cal); qh_cal = hi.predict(X_cal)
    ycal = y_cal.values if hasattr(y_cal, "values") else np.asarray(y_cal)
    scores = np.maximum(ql_cal - ycal, ycal - qh_cal)
    qhat = q_conformal(scores, alpha, conservative=CONSERVATIVE_Q)

    # refit em train+cal com #árvores selecionadas
    X_tc = pd.concat([X_tr, X_cal], axis=0)
    y_tc = pd.concat([y_tr, y_cal], axis=0) if hasattr(y_tr, "values") else np.concatenate([y_tr, y_cal])

    lo2 = XGBRegressor(objective="reg:quantileerror", quantile_alpha=q_lo,
                       **{**base_def, "n_estimators": n_lo})
    hi2 = XGBRegressor(objective="reg:quantileerror", quantile_alpha=q_hi,
                       **{**base_def, "n_estimators": n_hi})
    lo2.fit(X_tc, y_tc); ql_te = lo2.predict(X_te)
    hi2.fit(X_tc, y_tc); qh_te = hi2.predict(X_te)

    # ponto = q50 ou média dos quantis
    if use_q50_point:
        mid = XGBRegressor(objective="reg:quantileerror", quantile_alpha=0.5,
                           **{**base_def, "n_estimators": max(n_lo, n_hi)})
        mid.fit(X_tc, y_tc); y_point = mid.predict(X_te)
    else:
        y_point = 0.5*(ql_te + qh_te)

    return y_point, ql_te - qhat, qh_te + qhat, {"qhat": float(qhat), "n_lo": n_lo, "n_hi": n_hi}

def pi_enbpi_xgb(
    X_tr, y_tr, X_cal, y_cal, X_te,
    alpha=ALPHA, seed=SEED,
    base_params=None, n_estimators_ens=30, max_samples=0.8
):
    if not HAVE_XGB:
        raise RuntimeError("XGBoost não disponível.")
    from xgboost import XGBRegressor

    base_def = dict(**XGB_PARAMS_POINT, random_state=seed)
    if base_params: base_def.update(base_params)

    base = XGBRegressor(objective="reg:squarederror", **base_def)
    ensemble = BaggingRegressor(
        estimator=base, n_estimators=n_estimators_ens, max_samples=max_samples,
        bootstrap=True, n_jobs=-1, random_state=seed
    )
    ensemble.fit(X_tr, y_tr)
    y_cal_pred = ensemble.predict(X_cal)
    ycal = y_cal.values if hasattr(y_cal, "values") else np.asarray(y_cal)
    residuals = np.abs(ycal - y_cal_pred)
    q = q_conformal(residuals, alpha, conservative=CONSERVATIVE_Q)
    y_te_pred = ensemble.predict(X_te)
    return y_te_pred, y_te_pred - q, y_te_pred + q, dict(q_hat=float(q), n_estimators_ens=n_estimators_ens)

# =========================
# Métricas
# =========================
def interval_metrics(y_true, y_pred, lower, upper, alpha=ALPHA):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    lower = np.asarray(lower);   upper = np.asarray(upper)
    cover = np.mean((y_true >= lower) & (y_true <= upper))
    width = np.mean(np.maximum(0, upper - lower))
    eta = 10
    gamma = 0 if cover >= (1 - alpha) else 1
    cwc = width * (1 + gamma * np.exp(-eta * (cover - (1 - alpha))))
    return dict(Coverage=float(cover), Width=float(width), CWC=float(cwc),
                RMSE=float(rmse(y_true, y_pred)),
                MAE=float(mean_absolute_error(y_true, y_pred)),
                R2=float(r2_score(y_true, y_pred)))

# =========================
# Plot
# =========================
def plot_series(y_true, y_pred, lower, upper, title, outfile):
    plt.figure(figsize=(12,5), dpi=150)
    idx = np.arange(len(y_true))
    plt.plot(idx, y_true, label="Real",    lw=2.0, color=COLOR_REAL)
    plt.plot(idx, y_pred, label="Predito", lw=1.8, color=COLOR_PRED)
    plt.fill_between(idx, lower, upper, alpha=0.25, color=COLOR_FILL, label="Intervalo")
    plt.xlabel("Índice (tempo)")
    plt.ylabel("SiO₂ in concentrate (%)")
    plt.title(title)
    plt.legend(frameon=True)
    plt.tight_layout(); plt.grid(False)
    plt.savefig(outfile, bbox_inches="tight"); plt.close()

# =========================
# Main
# =========================
METHOD_CHOICES = {
    "t_student": "t_student",
    "full_conformal": "full_conformal_cvplus",
    "full_conformal_f": "full_conformal_f",
    "split_conformal": "split_conformal",
    "bootstrap": "bootstrap_residual",
    "bootstrap_f": "bootstrap_f",
    "cqr_xgb": "cqr_xgb",
    "enbpi_xgb": "enbpi_xgb",
}

def main():
    df = pd.read_csv(CSV_FILE, low_memory=False)
    missing = [c for c in FEATURES + [TARGET] if c not in df.columns]
    if missing:
        raise ValueError(f"Colunas faltando no CSV: {missing}")

    if "Data" in df.columns:
        df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
        df = df.sort_values("Data", kind="stable").reset_index(drop=True)

    X_full = ensure_numeric(df[FEATURES])
    y_full = pd.to_numeric(df[TARGET], errors="coerce")

    mask_valid = (~X_full.isna().any(axis=1)) & (~y_full.isna())
    X_full = X_full.loc[mask_valid].reset_index(drop=True)
    y_full = y_full.loc[mask_valid].reset_index(drop=True)

    # split primeiro (evita vazamento)
    X_tr, X_ca, X_te, y_tr, y_ca, y_te = split_data(X_full, y_full, SPLIT_TYPE, SEED, 0.80, 0.10)

    # outliers somente no treino
    if USE_ZSCORE_OUTLIERS:
        mu, sd = y_tr.mean(), (y_tr.std() if y_tr.std() > 0 else 1.0)
        keep_tr = ((y_tr - mu).abs() / sd) <= ZS_THRESH
        X_tr = X_tr.loc[keep_tr].reset_index(drop=True)
        y_tr = y_tr.loc[keep_tr].reset_index(drop=True)

    X_trcal = pd.concat([X_tr, X_ca], axis=0); y_trcal = pd.concat([y_tr, y_ca], axis=0)

    interval_model = get_point_model(MODEL_NAME, SEED)
    point_model    = get_point_model(MODEL_NAME, SEED)

    # ponto separado (opcional)
    y_point_te = None
    if USE_SEPARATE_POINT_MODEL:
        point_model.fit(X_tr, y_tr)
        y_point_te = point_model.predict(X_te)

    method_key = METHOD_CHOICES.get(METHOD, None)
    if method_key is None:
        raise ValueError(f"METHOD inválido: {METHOD}. Opções: {sorted(METHOD_CHOICES.keys())}")

    info = {}
    # ================= Dispatch =================
    if method_key == "t_student":
        m = clone(interval_model); m.fit(X_trcal, y_trcal)
        ypred = m.predict(X_te)
        res_tr = y_trcal.values - m.predict(X_trcal)
        lo, up, extra = t_student_intervals(ypred, res_tr, n_train=len(X_trcal), p=X_trcal.shape[1], alpha=ALPHA)
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "t-Student (approx.)"

    elif method_key == "full_conformal_cvplus":
        ypred, lo, up, extra = full_conformal_cvplus(interval_model, X_trcal, y_trcal, X_te, ALPHA, folds=5, seed=SEED)
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "Full Conformal (CV+)"

    elif method_key == "full_conformal_f":
        ypred, lo, up, extra = full_conformal_prediction_intervalsF(interval_model, X_trcal, X_te, y_trcal, alpha=ALPHA)
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "Full Conformal"

    elif method_key == "split_conformal":
        ypred, lo, up, extra = split_conformal_intervals(interval_model, X_tr, y_tr, X_ca, y_ca, X_te, ALPHA)
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "Split Conformal"

    elif method_key == "bootstrap_residual":
      ypred, lo, up, extra = bootstrap_residual_intervals(
          interval_model, X_tr, y_tr, X_ca, y_ca, X_te,
          n_bootstrap=1000, alpha=ALPHA, seed=SEED
      )
      info.update(extra)
      y_point = ypred if y_point_te is None else y_point_te
      title = "Residual Bootstrap (cal OOS)"


    elif method_key == "bootstrap_f":
        ypred, lo, up, extra = bootstrap_prediction_intervalsF(interval_model, X_tr, y_tr, X_te, n_bootstrap=1000, alpha=ALPHA, seed=SEED)
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "Residual Bootstrap"


    elif method_key == "ngboost_f":
        ypred, lo, up, extra = pi_ngboostF(X_tr, y_tr, X_te, ALPHA, SEED)
        info.update(extra); y_point = ypred if y_point_te is None else y_point_te
        title = "NGBoost (Normal-F)"

    elif method_key == "cqr_xgb":
        ypred, lo, up, extra = pi_cqr_xgb(
            X_tr, y_tr, X_ca, y_ca, X_te, ALPHA, SEED,
            xgb_params=XGB_PARAMS_QUANTILE,
            use_q50_point=True
        )
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "CQR (XGB Quantile)"

    elif method_key == "enbpi_xgb":
        ypred, lo, up, extra = pi_enbpi_xgb(
            X_tr, y_tr, X_ca, y_ca, X_te, ALPHA, SEED,
            base_params=XGB_PARAMS_POINT
        )
        info.update(extra)
        y_point = ypred if y_point_te is None else y_point_te
        title = "EnbPI (XGB)"

    else:
        raise ValueError("METHOD inválido (dispatch não alcançado).")

    mets = interval_metrics(y_te.values, y_point, lo, up, alpha=ALPHA)

    print("\n===== CONFIG =====")
    print(f"CSV_FILE={CSV_FILE}")
    print(f"SPLIT_TYPE={SPLIT_TYPE} | MODEL_NAME={MODEL_NAME} | METHOD={METHOD}")
    print(f"USE_ZSCORE_OUTLIERS={int(USE_ZSCORE_OUTLIERS)} (thr={ZS_THRESH}) | CONSERVATIVE_Q={int(CONSERVATIVE_Q)}")
    print(f"USE_SEPARATE_POINT_MODEL={int(USE_SEPARATE_POINT_MODEL)}")
    print("\n===== MÉTRICAS =====")
    for k, v in mets.items(): print(f"{k}: {v:.6f}")

    if info:
        print("\n===== INFO DO MÉTODO =====")
        for k, v in info.items():
            try: print(f"{k}: {float(v):.6f}")
            except: print(f"{k}: {v}")

    out_csv = RESULTS_DIR / f"series_{METHOD}_{SPLIT_TYPE}_{MODEL_NAME}.csv"
    pd.DataFrame({"y_true": y_te.values, "y_pred": y_point, "lower": lo, "upper": up}).to_csv(out_csv, index=False)
    print(f"\nSéries (teste) salvas em: {out_csv}")

    out_png = RESULTS_DIR / f"interval_{METHOD}_{SPLIT_TYPE}_{MODEL_NAME}.png"
    plot_series(y_true=y_te.values, y_pred=y_point, lower=lo, upper=up,
                title=f"{title} | α={ALPHA:.2f} | split={SPLIT_TYPE} | model={MODEL_NAME}",
                outfile=str(out_png))
    print(f"Figura salva em: {out_png}")

if __name__ == "__main__":
    main()


Overwriting Prediction_intervals.py


In [16]:
# !DATA_FILE=dados_2h_aggregado_final.csv \
# SPLIT_TYPE=random MODEL_NAME=xgb INTERVAL_METHOD=cqr_xgb ALPHA=0.05 SEED=42 \
# python Ray_flootation4.py


In [1]:
# run_all_methods_Prediction_intervals.py
# -*- coding: utf-8 -*-
import os
import time
import math
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# =======================
# CONFIG
# =======================
NUM_RUNS    = int(os.environ.get("NUM_RUNS", "20"))
MAIN_SCRIPT = os.environ.get("MAIN_SCRIPT", "Prediction_intervals.py")

RESULTS_DIR = Path("results")
AGG_DIR     = Path("aggregated_results")
AGG_DIR.mkdir(parents=True, exist_ok=True)

DATA_FILE   = os.environ.get("DATA_FILE", "dados_2h_aggregado_final.csv")
SPLIT_TYPE  = os.environ.get("SPLIT_TYPE", "random")          # random | temporal
MODEL_NAME  = os.environ.get("MODEL_NAME", "xgb")             # xgb | lgbm
ALPHA       = float(os.environ.get("ALPHA", "0.05"))

# Rode todos (padrão) ou filtre via env ONLY_METHODS="enbpi_rf,rf_split,cqr_gbrt"
ALL_METHODS = [
    # "t_student",
    "full_conformal",
    "full_conformal_f",
    "bootstrap_f",
    "bootstrap",
    "split_conformal",
    "cqr_xgb",
    "enbpi_xgb",
]

ONLY = os.environ.get("ONLY_METHODS", "").strip()
METHODS = [m.strip() for m in ONLY.split(",") if m.strip()] if ONLY else ALL_METHODS


DISPLAY = {
    # "t_student":      "T-Student",
    "full_conformal_f":"Full Conformal",
    # "full_conformal": "Full Conformal CV+",
    "bootstrap":      "Bootstrap",
    "split_conformal":"Split Conformal",
    "ngboost":        "NGBoost",
    "cqr_xgb":        "CQR",
    "enbpi_xgb":      "EnbPI",
}

# =======================
# Métricas a partir das séries
# =======================
def rmse(y_true, y_pred):
    from sklearn.metrics import mean_squared_error
    return math.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    from sklearn.metrics import mean_absolute_error
    return mean_absolute_error(y_true, y_pred)

def r2(y_true, y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_true, y_pred)

def coverage(y, lo, up):
    y = np.asarray(y); lo = np.asarray(lo); up = np.asarray(up)
    return float(np.mean((y >= lo) & (y <= up)))

def width_mean(lo, up):
    lo = np.asarray(lo); up = np.asarray(up)
    return float(np.mean(np.maximum(0.0, up - lo)))

def winkler_score(y, lo, up, alpha):
    """Winkler score (quanto menor, melhor)."""
    y = np.asarray(y); lo = np.asarray(lo); up = np.asarray(up)
    w = np.maximum(0.0, up - lo)
    below = y < lo
    above = y > up
    pen = np.zeros_like(w)
    pen[below] = (2.0 / alpha) * (lo[below] - y[below])
    pen[above] = (2.0 / alpha) * (y[above] - up[above])
    return float(np.mean(w + pen))

def cwc(cover, width, alpha, eta=10.0):
    gamma = 0.0 if cover >= (1 - alpha) else 1.0
    return float(width * (1 + gamma * math.exp(-eta * (cover - (1 - alpha)))))

def metrics_from_series(series_csv, alpha=0.05):
    df = pd.read_csv(series_csv)
    y   = df["y_true"].values
    yph = df["y_pred"].values
    lo  = df["lower"].values
    up  = df["upper"].values
    cov = coverage(y, lo, up)
    wid = width_mean(lo, up)
    win = winkler_score(y, lo, up, alpha)
    return dict(
        Coverage=cov,
        Width=wid,
        Winkler=win,
        CWC=cwc(cov, wid, alpha),
        RMSE=rmse(y, yph),
        MAE=mae(y, yph),
        R2=r2(y, yph),
    )

# =======================
# Helpers de plot (pastel)
# =======================
def pastel_colors(n, cmap_name="Pastel1"):
    cmap = cm.get_cmap(cmap_name, max(n, 3))
    return [cmap(i) for i in range(n)]

def boxplot_metric(df, metric, outpath, target_line=None, ylabel=None, title=None):
    methods = df["Unnamed: 0"].unique().tolist()
    data = [df.loc[df["Unnamed: 0"] == m, metric].dropna().values for m in methods]
    colors = pastel_colors(len(methods), "Pastel1")

    w = max(8, 1.1*len(methods))
    fig, ax = plt.subplots(figsize=(w, 5), dpi=150)
    bp = ax.boxplot(data, labels=methods, showfliers=True, patch_artist=True,
                    boxprops=dict(linewidth=1.0, edgecolor="0.3"),
                    whiskerprops=dict(linewidth=1.0, color="0.3"),
                    capprops=dict(linewidth=1.0, color="0.3"),
                    medianprops=dict(linewidth=1.2, color="black"))
    for patch, c in zip(bp["boxes"], colors):
        patch.set_facecolor(c)
    if target_line is not None:
        ax.axhline(target_line, linestyle="--", linewidth=1.2, color="tab:blue")
    ax.set_ylabel(ylabel or metric)
    ax.set_title(title or metric)
    plt.setp(ax.get_xticklabels(), rotation=20, ha="right")
    ax.grid(axis="y", linestyle="--", alpha=0.35)
    plt.tight_layout()
    plt.savefig(outpath, dpi=300, bbox_inches="tight")
    plt.close(fig)



def main():
    print(f"Rodando {NUM_RUNS} execuções de {MAIN_SCRIPT} para métodos: {', '.join(METHODS)}")
    rows = []

    for run in range(NUM_RUNS):
        print(f"\n=== RUN {run+1:02d}/{NUM_RUNS} ===")
        t_run0 = time.time()

        for method in METHODS:
            env = os.environ.copy()
            env["DATA_FILE"]       = DATA_FILE
            env["SPLIT_TYPE"]      = SPLIT_TYPE
            env["MODEL_NAME"]      = MODEL_NAME
            env["INTERVAL_METHOD"] = method
            env["ALPHA"]           = str(ALPHA)
            env["SEED"]            = str(42 + run)

            # mede o tempo do método
            t0 = time.time()
            proc = subprocess.run(["python", MAIN_SCRIPT], capture_output=True, text=True, env=env)
            method_dt = time.time() - t0

            if proc.returncode != 0:
                print(f"  -> {method}: ERRO — pulando. STDERR:\n{proc.stderr}")
                continue

            # série salva por ray_flotation4.py
            series_csv = RESULTS_DIR / f"series_{method}_{SPLIT_TYPE}_{MODEL_NAME}.csv"
            if not series_csv.exists():
                print(f"  -> {method}: série não encontrada ({series_csv}) — pulando.")
                continue

            mets = metrics_from_series(series_csv, alpha=ALPHA)

            # linha no formato solicitado (Unnamed: 0 = nome do método)
            rows.append({
                "Unnamed: 0": DISPLAY.get(method, method),
                "Coverage":   mets["Coverage"],
                "Width":      mets["Width"],
                "Winkler":    mets["Winkler"],
                "RMSE":       mets["RMSE"],
                "R2":         mets["R2"],
                "MAE":        mets["MAE"],
                "MethodTime_s": method_dt,
                "run_id":     run + 1,
                "execution_time": np.nan,
                "MethodCode": method,
                "Split": SPLIT_TYPE,
                "Model": MODEL_NAME,
            })
            print(f"  -> {DISPLAY.get(method, method)} | cov={mets['Coverage']:.3f} | width={mets['Width']:.3f} | t={method_dt:.2f}s")

        run_dt = time.time() - t_run0
        # preenche execução total deste run nas linhas correspondentes
        for r in rows:
            if np.isnan(r["execution_time"]) and r["run_id"] == (run + 1):
                r["execution_time"] = run_dt
        print(f"RUN {run+1:02d} concluído em {run_dt:.2f}s")

    if not rows:
        print("\nNenhuma execução bem-sucedida.")
        return

    # DataFrame final e salvamento no formato pedido
    df = pd.DataFrame(rows)

    # ordena colunas para bater com seu exemplo
    ordered_cols = [
        "Unnamed: 0","Coverage","Width","Winkler","RMSE","R2","MAE","MethodTime_s","run_id","execution_time"
    ]
    # anexa extras ao final (se existirem)
    extra_cols = [c for c in df.columns if c not in ordered_cols]
    df = df[ordered_cols + extra_cols]

    all_csv = AGG_DIR / "all_metrics.csv"
    df.to_csv(all_csv, index=False)
    print(f"\n[OK] salvo: {all_csv}")

    # =======================
    # Estatísticas agregadas
    # =======================
    agg = (df.groupby("Unnamed: 0")
             .agg({
                 "Coverage": ["mean","std","min","max"],
                 "Width":    ["mean","std","min","max"],
                 "Winkler":  ["mean","std","min","max"],
                 "RMSE":     ["mean","std","min","max"],
                 "R2":       ["mean","std","min","max"],
                 "MAE":      ["mean","std","min","max"],
                 "MethodTime_s": "mean",
                 "execution_time": "mean"
             })
             .round(4))
    agg.columns = ["_".join(c) if isinstance(c, tuple) else c for c in agg.columns]
    agg_csv = AGG_DIR / "aggregated_stats.csv"
    agg.to_csv(agg_csv)
    print(f"[OK] estatísticas: {agg_csv}")

    # porcentagem de execuções abaixo do alvo de cobertura
    below = (df.assign(below=lambda d: (d["Coverage"] < (1-ALPHA)).astype(float))
               .groupby("Unnamed: 0")["below"].mean() * 100).round(1)
    below.to_csv(AGG_DIR / "coverage_below_target_pct.csv")
    print("\n% de execuções com cobertura < alvo (por método):")
    print(below)


    boxplot_metric(df, "Coverage", AGG_DIR / "boxplot_coverage.png",
                   target_line=(1-ALPHA), ylabel="Coverage", title="Cobertura por Método")
    boxplot_metric(df, "Width",    AGG_DIR / "boxplot_width.png",
                   ylabel="Width", title="Largura do Intervalo por Método")
    boxplot_metric(df, "Winkler",  AGG_DIR / "boxplot_winkler.png",
                   ylabel="Winkler", title="Winkler Score por Método")
    boxplot_metric(df, "RMSE",     AGG_DIR / "boxplot_rmse.png",
                   ylabel="RMSE", title="RMSE por Método")
    boxplot_metric(df, "MAE",      AGG_DIR / "boxplot_mae.png",
                   ylabel="MAE", title="MAE por Método")


    print("\n[OK] Boxplots salvos em:", AGG_DIR)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'numpy'